## Crawling


In [1]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from turtle import title


def crawl(url, num_page):
    
    result = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    
    #find paging page 
    paging = soup.find_all("nav",{'class':'pagination'})
    paging_link = paging[0].find_all('a',{'class':'pagination__page'})
    last_page = num_page

    #looping through paging
    for i in range(1,last_page+1):
        print(url+'page/'+str(i))

        #find article link
        req = requests.get(url+'page/'+str(i))
        soup = BeautifulSoup(req.text, "lxml")
        news_links = soup.find_all("div",{'class':'thumb-container thumb-65'})
        # print(news_links)

        #looping through article link
    
        for idx,news in enumerate(news_links):
            news_dict = {}

            #find news title
            title_news= news.find('h1',{'class':'entry__title'}).text.replace("\n", "").lower()
            title_news = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', title_news)
            title_news = re.sub(r'[0-9]', '', title_news)
    

            #find urll news
            url_news = news.find('a',{'class':''}).get('href')

            
            #find news content in url
            req_news =  requests.get(url_news)
            soup_news = BeautifulSoup(req_news.text, "lxml")

            #find news content 
            news_content = soup_news.find("div",{'class':'entry__article-wrap'})

            #find paragraph in news content 
            p = news_content.find_all('p')
            content = ' '.join(item .text for item in p)
            news_content = content.encode('utf8','replace')

            #wrap in dictionary 
            news_dict['no']=idx+1
            news_dict['url'] = url_news
            news_dict['title'] = title_news
            news_dict['content'] = news_content
            result.append(news_dict)
         
    return result

In [3]:
url = 'https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/'
crwl  = crawl(url, 4)

https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/1
https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/2
https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/3
https://visitjawatengah.jatengprov.go.id/id/regency/kabupaten-semarang/destinasi-wisata/page/4


In [4]:
crwl = pd.DataFrame(crwl)
crwl.head()

,no,url,title,content
0,1,https://visitjawatengah.jatengprov.go.id/id/re...,puri kayana camping ground,b'#SobatWisata\xc2\xa0kangen ngecamp nggak nih...
1,2,https://visitjawatengah.jatengprov.go.id/id/re...,mengunjungi wisata religi nyatnyono,b'Desa Nyatnyono di lereng Gunung Ungaran dike...
2,3,https://visitjawatengah.jatengprov.go.id/id/re...,candi dukuh petilasan prabu brawijaya,"b'Candi Dukuh terletak di Desa Rowoboni, Kecam..."
3,4,https://visitjawatengah.jatengprov.go.id/id/re...,hal spesial di desa wisata lerep,b'Jalan-jalan ke Semarang? Mampirlah ke Desa W...
4,5,https://visitjawatengah.jatengprov.go.id/id/re...,all in one in sunrise hill gedong songo,b'#SobatWisata\xc2\xa0lagi cari referensi rest...


In [5]:
crwl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   no       38 non-null     int64 
 1   url      38 non-null     object
 2   title    38 non-null     object
 3   content  38 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.3+ KB


In [6]:
def documents_cleaning(doc):
  docs_clean = []

  for d in doc:
      html = d.decode("utf-8") 
      document_test = re.sub(r'[^\x00-\x7F]+', ' ', html)
      document_test = re.sub(r'@\w+', '', document_test)
      document_test = document_test.lower()
      document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
      document_test = re.sub(r'[0-9]', '', document_test)
      document_test = re.sub(r'\s{2,}', ' ', document_test)
      document_test = document_test.replace("www", " ")
      document_test = document_test.replace(" com ", " ")
      document_test = document_test.replace("kompas", " ")
      document_test = document_test.replace("visitjateng", " ")
      document_test = document_test.replace("visitjawatengah", " ")
      document_test = document_test.replace("kabsemarangtourism", " ")
      document_test = document_test.replace("html", " ")
      document_test = document_test.replace("http", " ")
      document_test = document_test.replace("https", " ")
      docs_clean.append(document_test)

  return docs_clean

In [7]:
crwl['clean_content'] = documents_cleaning(crwl['content'])

In [8]:
crwl.head()



,no,url,title,content,clean_content
0,1,https://visitjawatengah.jatengprov.go.id/id/re...,puri kayana camping ground,b'#SobatWisata\xc2\xa0kangen ngecamp nggak nih...,sobatwisata kangen ngecamp nggak nih cuss ber...
1,2,https://visitjawatengah.jatengprov.go.id/id/re...,mengunjungi wisata religi nyatnyono,b'Desa Nyatnyono di lereng Gunung Ungaran dike...,desa nyatnyono di lereng gunung ungaran dikena...
2,3,https://visitjawatengah.jatengprov.go.id/id/re...,candi dukuh petilasan prabu brawijaya,"b'Candi Dukuh terletak di Desa Rowoboni, Kecam...",candi dukuh terletak di desa rowoboni kecamata...
3,4,https://visitjawatengah.jatengprov.go.id/id/re...,hal spesial di desa wisata lerep,b'Jalan-jalan ke Semarang? Mampirlah ke Desa W...,jalan jalan ke semarang mampirlah ke desa wisa...
4,5,https://visitjawatengah.jatengprov.go.id/id/re...,all in one in sunrise hill gedong songo,b'#SobatWisata\xc2\xa0lagi cari referensi rest...,sobatwisata lagi cari referensi resto pengina...


In [10]:
datalink = crwl[['no','title','content','clean_content','url']]
datalink.to_csv('crawl_wisata.csv', index=False)
datalink.head()

,no,title,content,clean_content,url
0,1,puri kayana camping ground,b'#SobatWisata\xc2\xa0kangen ngecamp nggak nih...,sobatwisata kangen ngecamp nggak nih cuss ber...,https://visitjawatengah.jatengprov.go.id/id/re...
1,2,mengunjungi wisata religi nyatnyono,b'Desa Nyatnyono di lereng Gunung Ungaran dike...,desa nyatnyono di lereng gunung ungaran dikena...,https://visitjawatengah.jatengprov.go.id/id/re...
2,3,candi dukuh petilasan prabu brawijaya,"b'Candi Dukuh terletak di Desa Rowoboni, Kecam...",candi dukuh terletak di desa rowoboni kecamata...,https://visitjawatengah.jatengprov.go.id/id/re...
3,4,hal spesial di desa wisata lerep,b'Jalan-jalan ke Semarang? Mampirlah ke Desa W...,jalan jalan ke semarang mampirlah ke desa wisa...,https://visitjawatengah.jatengprov.go.id/id/re...
4,5,all in one in sunrise hill gedong songo,b'#SobatWisata\xc2\xa0lagi cari referensi rest...,sobatwisata lagi cari referensi resto pengina...,https://visitjawatengah.jatengprov.go.id/id/re...
